In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e6:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73290%2F8710574%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240620%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240620T082604Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3f314103e87b6a9898f4d77a8ad693f020eb19223a23d0635e3d3020ff42861e89da46f7a5f31159a57f8fc16232617a3b40e52095dc924239bfe70d9096820a390a43e736d81617ec84c2ea65e55658c725d536c1d7feb9c2839d7b047cbc35e47d6079716e7658b24a69d529d767e85043165387e54705960514f4714f7eb5f7c90f45fc5d71b44115edd9ceaa021cf4532dc5947245b5a98fcebaa755f5bc2fc575086ba30364c2ae2e4d03f31965740be6c03bc351788faf63f52715449b16d5c0056644260d2dbab5040b9b88ff3542b620eddd0fc23cb57437bc3fd39868b5aea1b7e5424de60ed5fb0ae07ed1ee73bb9b96f786f4b1c9b999882a704e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
test_df=pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")
train_df

In [ ]:
test_df

In [ ]:
from sklearn.preprocessing import LabelEncoder

# create an instance of LabelEncoder
le = LabelEncoder()

# apply the fit_transform method to the target variable
train_df["Target"] = le.fit_transform(train_df["Target"])

In [ ]:
train_df

In [ ]:
X = train_df.drop(['Target'], axis=1)
y = train_df[['Target']]
X

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming X and y are your features and labels
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# Create a Decision Tree Classifier with specific hyperparameters
clf = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=2, min_samples_leaf=12)

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming X and y are your features and labels

# If y is a DataFrame or Series, convert it to a 1D array
if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
    y = y.values.ravel()  # Convert 'y' to a 1D array if it's a DataFrame or Series

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a Random Forest Classifier with specific hyperparameters
rf_clf = RandomForestClassifier(
    n_estimators=200,             # Number of trees in the forest
    criterion='entropy',          # Function to measure the quality of a split
    max_depth=10,                 # Maximum depth of the tree
    min_samples_split=2,          # Minimum number of samples required to split an internal node
    min_samples_leaf=12,          # Minimum number of samples required to be at a leaf node
    random_state=42               # Seed for reproducibility
)

# Train the model
rf_clf.fit(X_train, y_train)

# Make predictions
y_pred = rf_clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Assuming X and y are your features and labels

# Ensure y is a 1D array if it is a DataFrame or Series
if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
    y = y.values.ravel()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create an XGBoost Classifier with specific hyperparameters
xgb_clf = XGBClassifier(
    n_estimators=200,             # Number of trees in the forest
    max_depth=10,                 # Maximum depth of the tree
    learning_rate=0.1,            # Learning rate
    subsample=0.8,                # Subsample ratio of the training instances
    colsample_bytree=0.8,         # Subsample ratio of columns when constructing each tree
    gamma=0,                      # Minimum loss reduction required to make a further partition
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'         # Evaluation metric for early stopping
)

# Train the model
xgb_clf.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_clf.predict(X_test)

# Evaluate the model
print("Accuracy (XGBoost):", accuracy_score(y_test, y_pred_xgb))
print("Classification Report (XGBoost):")
print(classification_report(y_test, y_pred_xgb))


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Assuming X and y are your features and labels

# Ensure y is a 1D array if it is a DataFrame or Series
if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
    y = y.values.ravel()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a CatBoost Classifier with specific hyperparameters
cat_clf = CatBoostClassifier(
    iterations=200,               # Number of boosting iterations
    depth=10,                     # Depth of the tree
    learning_rate=0.1,            # Learning rate
    l2_leaf_reg=3,                # L2 regularization coefficient
    random_state=42,
    verbose=0                     # Silent output
)

# Train the model
cat_clf.fit(X_train, y_train)

# Make predictions
y_pred_cat = cat_clf.predict(X_test)

# Evaluate the model
print("Accuracy (CatBoost):", accuracy_score(y_test, y_pred_cat))
print("Classification Report (CatBoost):")
print(classification_report(y_test, y_pred_cat))


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Assuming X and y are your features and labels

# Ensure y is a 1D array if it is a DataFrame or Series
if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
    y = y.values.ravel()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a LightGBM Classifier with specific hyperparameters
lgb_clf = lgb.LGBMClassifier(
    n_estimators=200,             # Number of boosting iterations
    max_depth=10,                 # Maximum depth of each tree
    learning_rate=0.1,            # Learning rate
    subsample=0.8,                # Subsample ratio of the training instances
    colsample_bytree=0.8,         # Subsample ratio of columns when constructing each tree
    random_state=42
)

# Train the model
lgb_clf.fit(X_train, y_train)

# Make predictions
y_pred_lgb = lgb_clf.predict(X_test)

# Evaluate the model
print("Accuracy (LightGBM):", accuracy_score(y_test, y_pred_lgb))
print("Classification Report (LightGBM):")
print(classification_report(y_test, y_pred_lgb))


In [ ]:
import lightgbm as lgb
import pandas as pd

# Make predictions using the LightGBM model
y_pred_lgb = lgb_clf.predict(test_df)

# Create a DataFrame for submission with 'id' and 'Target' columns
submission = pd.DataFrame({
    'id': test_df['id'],    # Assuming 'id' column exists in test_df
    'Target': y_pred_lgb    # Predicted labels
})

# Save the submission DataFrame to a CSV file without row indices
submission.to_csv('submission.csv', index=False)

# Optionally, display the first few rows of the submission DataFrame
print(submission.head())